In [2]:
# PANDAS IS FOR DATA WRANGLING
import pandas as pd

# IMPORT OUR LINEAR REGRESSION MODEL
from sklearn.linear_model import LinearRegression

# IMPORT OUR RANDOM FOREST REGERSSOR MODEL
from sklearn.ensemble import RandomForestRegressor


# IM LEAVING THERES HERE BECAUSE IM NICE
# METRICS HELP US SCORE OUR MODEL
from sklearn import metrics

# HELP US SPLIT OUR DATA INTO TESTING A TRAINING
from sklearn.model_selection import train_test_split

# HELPER FUNCTION WITH Z-SCORES TO REMOVE OUTLIERS
import scipy.stats as stats

%matplotlib inline

import pickle

### Load regression model 

In [3]:
filename = 'rf_regressor1.pkl'
model = pickle.load(open(filename, 'rb'))

#### Load category codes for non numerical cols 

In [104]:
city_cat_codes = pd.read_csv('city_cat_codes.csv')
stabbr_cat_codes = pd.read_csv('stabbr_cat_codes.csv')
accred_cat_codes = pd.read_csv('accred_cat_codes.csv')

In [12]:
# Read in basic inst data 
#Read in reduced institutional data 
na_vals = ['Na', 'NaN', 'PrivacySuppressed']
df = pd.read_csv('../data/reduced_inst_data_with_cats.csv', na_values=na_vals)

In [28]:
selected_features = ['UNITID', 'INSTNM', 'CITY', 'STABBR', 'ACCREDAGENCY', 'CONTROL', 'COSTT4_A', 'COSTT4_P',
       'TUITIONFEE_IN', 'TUITIONFEE_OUT', 'PFTFAC', 'PCTPELL']
df = df[selected_features]

In [30]:
df.head()

,UNITID,INSTNM,CITY,STABBR,ACCREDAGENCY,CONTROL,COSTT4_A,COSTT4_P,TUITIONFEE_IN,TUITIONFEE_OUT,PFTFAC,PCTPELL
0,100654,Alabama A & M University,Normal,AL,Southern Association of Colleges and Schools C...,1,22489.0,NaN,9744.0,18354.0,0.7411,0.7067
1,100663,University of Alabama at Birmingham,Birmingham,AL,Southern Association of Colleges and Schools C...,1,24347.0,NaN,8568.0,19704.0,0.7766,0.3632
2,100690,Amridge University,Montgomery,AL,Southern Association of Colleges and Schools C...,2,17680.0,NaN,6900.0,6900.0,1.0000,0.7673
3,100706,University of Alabama in Huntsville,Huntsville,AL,Southern Association of Colleges and Schools C...,1,23441.0,NaN,10714.0,22362.0,0.6544,0.2698
4,100724,Alabama State University,Montgomery,AL,Southern Association of Colleges and Schools C...,1,21476.0,NaN,11068.0,19396.0,0.5826,0.7448


In [31]:
df.duplicated(['INSTNM']).sum()

105

In [32]:
df[df.duplicated(['INSTNM'])]

,UNITID,INSTNM,CITY,STABBR,ACCREDAGENCY,CONTROL,COSTT4_A,COSTT4_P,TUITIONFEE_IN,TUITIONFEE_OUT,PFTFAC,PCTPELL
300,115001,Glendale Community College,Glendale,CA,Western Association of Schools and Colleges Ac...,1,11239.0,NaN,1175.0,6719.0,0.2899,0.3557
862,143491,Cannella School of Hair Design-Chicago,Chicago,IL,National Accrediting Commission of Career Arts...,3,NaN,NaN,NaN,NaN,NaN,0.6429
1169,155186,Highland Community College,Highland,KS,Higher Learning Commission,1,11528.0,NaN,3077.0,3625.0,1.0000,0.3095
1197,155900,Southwestern College,Winfield,KS,Higher Learning Commission,2,42702.0,NaN,30150.0,30150.0,1.0000,0.4115
1440,165671,Emmanuel College,Boston,MA,New England Commission on Higher Education,2,55739.0,NaN,39804.0,39804.0,0.4724,0.2614
...,...,...,...,...,...,...,...,...,...,...,...,...
6249,21429001,New Castle School of Trades,Pulaski,PA,Accrediting Commission of Career Schools and C...,3,NaN,NaN,NaN,NaN,NaN,NaN
6333,42312001,Compu-Med Vocational Careers Corp,Miami,FL,Accrediting Commission of Career Schools and C...,3,NaN,NaN,NaN,NaN,NaN,NaN
6366,44936601,Make-up Designory,New York,NY,Accrediting Commission of Career Schools and C...,3,NaN,NaN,NaN,NaN,NaN,NaN
6383,45486501,Cambridge College of Healthcare & Technology,Atlanta,GA,Accrediting Bureau of Health Education Schools,3,NaN,NaN,14827.0,NaN,NaN,NaN


In [33]:
df[df.INSTNM=='Glendale Community College']

,UNITID,INSTNM,CITY,STABBR,ACCREDAGENCY,CONTROL,COSTT4_A,COSTT4_P,TUITIONFEE_IN,TUITIONFEE_OUT,PFTFAC,PCTPELL
83,104708,Glendale Community College,Glendale,AZ,Higher Learning Commission,1,11843.0,NaN,2070.0,7854.0,1.0000,0.3292
300,115001,Glendale Community College,Glendale,CA,Western Association of Schools and Colleges Ac...,1,11239.0,NaN,1175.0,6719.0,0.2899,0.3557


In [34]:
df.duplicated(['UNITID']).sum()

0

In [35]:
df.duplicated(['INSTNM', 'CITY', 'STABBR']).sum()

8

In [36]:
df[df.duplicated(['INSTNM', 'CITY', 'STABBR'])]

,UNITID,INSTNM,CITY,STABBR,ACCREDAGENCY,CONTROL,COSTT4_A,COSTT4_P,TUITIONFEE_IN,TUITIONFEE_OUT,PFTFAC,PCTPELL
862,143491,Cannella School of Hair Design-Chicago,Chicago,IL,National Accrediting Commission of Career Arts...,3,NaN,NaN,NaN,NaN,NaN,0.6429
3219,224679,Western Technical College,El Paso,TX,Accrediting Commission of Career Schools and C...,3,NaN,30502.0,NaN,NaN,0.6437,0.5952
4590,443696,Interactive College of Technology,Houston,TX,Council on Occupational Education,3,18225.0,NaN,9530.0,9530.0,NaN,0.7778
4845,450711,Trend Barber College,Houston,TX,Council on Occupational Education,3,NaN,13045.0,NaN,NaN,NaN,0.4964
5365,476063,Tulsa Welding School-Jacksonville,Jacksonville,FL,Accrediting Commission of Career Schools and C...,3,NaN,26405.0,NaN,NaN,NaN,0.4792
5512,482015,Southern Careers Institute-San Antonio,San Antonio,TX,Council on Occupational Education,3,NaN,29964.0,NaN,NaN,0.5417,0.5954
6230,19857001,Gaston College,Dallas,NC,Southern Association of Colleges and Schools C...,1,NaN,NaN,2704.0,8848.0,NaN,NaN
6404,45920401,Unitek College,Fremont,CA,Accrediting Commission of Career Schools and C...,3,NaN,NaN,33559.0,NaN,NaN,NaN


In [58]:
df[df.INSTNM == 'Cannella School of Hair Design-Chicago']

,UNITID,INSTNM,CITY,STABBR,ACCREDAGENCY,CONTROL,COSTT4_A,COSTT4_P,TUITIONFEE_IN,TUITIONFEE_OUT,PFTFAC,PCTPELL
861,143482,Cannella School of Hair Design-Chicago,Chicago,IL,National Accrediting Commission of Career Arts...,3,NaN,13690.0,NaN,NaN,NaN,0.5789
862,143491,Cannella School of Hair Design-Chicago,Chicago,IL,National Accrediting Commission of Career Arts...,3,NaN,NaN,NaN,NaN,NaN,0.6429


In [59]:
#Since these appear to be the same institutions, will drop duplicates, keeping last occurence 
df.drop_duplicates(['INSTNM', 'CITY', 'STABBR'], keep='last', inplace=True)

In [62]:
#Sanity check 
df.duplicated(['INSTNM', 'CITY', 'STABBR']).sum()

0

In [24]:
# Fill nulls in X numeric columns with median, add column indicating if value was missing 
for label, content in df.items(): 
    if pd.api.types.is_numeric_dtype(content): 
        if pd.isnull(content).sum(): 
            #Add column that has true if row had missing value for the column 
            df[label+'_is_missing'] = pd.isnull(content) 
            # Fill missing values for median for the column 
            df[label] = content.fillna(content.median())

## Logic for taking new input and feeding it to model for prediction

In [63]:
predictive_features = ['UNITID', 'CITY', 'STABBR', 'ACCREDAGENCY', 'CONTROL', 'COSTT4_A', 'COSTT4_P',
       'TUITIONFEE_IN', 'TUITIONFEE_OUT', 'PFTFAC', 'PCTPELL']

In [64]:
sample_input = {'INSTNM':'Glendale Community College', 'CITY': 'Glendale', 'STABBR': 'AZ'} 

In [112]:
condition = (df.INSTNM == sample_input['INSTNM']) & (df.CITY == sample_input['CITY']) & (df.STABBR == sample_input['STABBR'])
inst_data = df[condition] 

In [113]:
inst_data

,UNITID,INSTNM,CITY,STABBR,ACCREDAGENCY,CONTROL,COSTT4_A,COSTT4_P,TUITIONFEE_IN,TUITIONFEE_OUT,PFTFAC,PCTPELL
83,104708,Glendale Community College,Glendale,AZ,Higher Learning Commission,1,11843.0,NaN,2070.0,7854.0,1.0,0.3292


In [115]:
print(inst_data.STABBR.values[0])

AZ


In [117]:
stabbr_cat_codes[stabbr_cat_codes.STABBR == 'AZ'].STABBR_code.values[0]

4

In [135]:
# Function for filling in numeric nulls 
def fill_na_numeric(dataframe, columns, df_full): 
    for label, content in dataframe.items(): 
        if pd.api.types.is_numeric_dtype(content) and label in columns:
            # Add column that has true if row had missing value for the column 
            dataframe[label+'_is_missing'] = pd.isna(content)
            if pd.isnull(content).sum():  
                # Fill missing values for median for the column 
                dataframe[label] = content.fillna(df_full[label].median())

In [111]:
stabbr_cat_codes

,STABBR,STABBR_code
0,AK,0
1,AL,1
2,AR,2
3,AS,3
4,AZ,4
5,CA,5
6,CO,6
7,CT,7
8,DC,8
9,DE,9


In [118]:
# Function for turning categories into numeric 
def fill_cat_nums(df):
    df['STABBR'] = stabbr_cat_codes[stabbr_cat_codes.STABBR == df.STABBR.values[0]].STABBR_code.values[0]
    df['ACCREDAGENCY'] = accred_cat_codes[accred_cat_codes.ACCREDAGENCY == df['ACCREDAGENCY'].values[0]].ACCREDAGENCY_code.values[0]
    df['CITY'] = city_cat_codes[city_cat_codes.CITY == df['CITY'].values[0]].CITY_code.values[0]

In [137]:
# gather necessary data for ml X using input
def ml_input(predictive_features, user_input, df_full): 
    # filter df for matching institution
    condition = (df_full.INSTNM == user_input['INSTNM']) & (df_full.CITY == user_input['CITY']) & (df_full.STABBR == user_input['STABBR'])
    inst_data = df_full[condition] 
    # Reduce inst_data df to keep only predictive_features
    inst_data = inst_data[predictive_features]
    # fill null values if any in returned data
    fill_na_numeric(inst_data, inst_data.columns.drop(['UNITID', 'CONTROL']) , df_full)
    #Turn categories to nums 
    fill_cat_nums(inst_data)
    return inst_data
    
    
            

In [138]:
my_input = ml_input(predictive_features, sample_input, df)

In [139]:
my_input

,UNITID,CITY,STABBR,ACCREDAGENCY,CONTROL,COSTT4_A,COSTT4_P,TUITIONFEE_IN,TUITIONFEE_OUT,PFTFAC,PCTPELL,COSTT4_A_is_missing,COSTT4_P_is_missing,TUITIONFEE_IN_is_missing,TUITIONFEE_OUT_is_missing,PFTFAC_is_missing,PCTPELL_is_missing
83,104708,798,4,20,1,11843.0,19387.5,2070.0,7854.0,1.0,0.3292,False,True,False,False,False,False


In [143]:
prediction = model.predict(my_input)

In [150]:
prediction

array([8544.455])

In [148]:
round(prediction[0], 2)

8544.46